# NLTK and cleaning


In [ ]:
import json

import matplotlib

import nltk

: 

In [ ]:
with open("data/train/00001756c60be8.json",'r') as f: 
   jsondict = json.load(f)  

: 

In [ ]:
with open("data/train/0a06dfd3d34f34.json",'r') as f: 
   jsondict2 = json.load(f)  

: 

In [ ]:
print(jsondict["source"]['1862f0a6'])

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


: 

In [ ]:
jsondict["source"].keys()

dict_keys(['1862f0a6', '2a9e43d6', '038b763d', '2eefe0ef', '0beab1cd', '9a78ab76', 'ebe125d5', 'd9dced8b', '86497fe1', 'e2c8e725', 'ff7c44ed', '0e7c906e', 'dd0c804a', '781bbf3c', 'bd94f005', '62638fba', 'bb69e88c', '6b5664c7', '23783525', '8522781a', '8ca8392c', '17ec3fc4', '76512d50', 'a98c5d9f', '06365725', '59959af5', '80151ab7', '5bf9ca51', 'f5504853', '9f50dca0', '21616367', 'fcb6792d', '63c26fa2', '4bb2e30a', 'a6357f7e', '45082c89', '77e56113', '448eb224', '032e2820', '8554b284', '36002912', 'ac301a84', '23705731', '1496beaf', '2e1a5949', '7e2f170a', 'bfbde93e', '0d136e08', '915643b3', '8ffe0b25', '8a4c95d1', 'b69a4f9b', 'c3ce0945', '3eebeb87', '1ae087ab', 'aaad8355', '503926eb', '3e5f860d'])

: 

In [ ]:
jsondict2["source"].keys()

dict_keys(['f18fe90f', '7b146807', '7fc42e3d', 'e4187045', 'f3f1f1ca', '64963778', '64fe4e2b', '707dcebc', '98f64a54', 'b5d0e650', 'a332db8d', '214b3983', 'b24b25fc', '5b3482ba', '948395b6', '61b20a7e', '45258e3a', '4f0c6a57', '1ac84d33', '53238e89', '632ba6b6', '81238c6e', 'c8f8caee', '5ed60d72', 'a44a736b', 'e0fa466d', 'fb7c7020', 'a5b26c3f', '280ac2bc', '826a3eb5', '6fa0283e', '8ca1a048', '8ca9d124', '69220509', 'fd1cac48', '57db3379', 'fb23cc8a', '43c994f4', '31203890', '0b730456', '0950a6f1', 'e2ebae3a', '3f658999', '208b4af1', 'dc293fd5', '3ab72d73', 'dc4c3dda', 'a48b5ead', 'f942541a', '33bc1f76', '69556613', '22ca3e04', '41b07cec', 'aeadecf5', 'bded11a2', '9673ab13', '3b249172', 'b95dfce2', 'e9cc8f46', 'ac401605', '37f975f7'])

: 

In [ ]:
testtext1 = jsondict["source"]['aaad8355']
testtext1

'*Тип данных обучающего сета*'

: 

In [ ]:
testtext2 = jsondict2["source"]['b95dfce2']
testtext2

'## The Goal\n\nThe Source for this competition is: \n\nhttps://www.kaggle.com/c/learn-together/data\n\nThe study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. Each observation is a 30m x 30m patch. Our goal is to predict an integer classification for the forest cover type. The seven types are:\n\n1 - Spruce/Fir\n\n2 - Lodgepole Pine\n\n3 - Ponderosa Pine\n\n4 - Cottonwood/Willow\n\n5 - Aspen\n\n6 - Douglas-fir\n\n7 - Krummholz\n\nThe training set (15120 observations) contains both features and the Cover_Type. The test set contains only the features. You must predict the Cover_Type for every row in the test set (565892 observations).'

: 

# Try out NLTK

tokenize

In [ ]:
import re
import pprint

: 

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kak/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

: 

In [ ]:

text = re.sub(r"[^a-zA-Z0-9]", " ", testtext2.lower())
text

'   the goal  the source for this competition is    https   www kaggle com c learn together data  the study area includes four wilderness areas located in the roosevelt national forest of northern colorado  each observation is a 30m x 30m patch  our goal is to predict an integer classification for the forest cover type  the seven types are   1   spruce fir  2   lodgepole pine  3   ponderosa pine  4   cottonwood willow  5   aspen  6   douglas fir  7   krummholz  the training set  15120 observations  contains both features and the cover type  the test set contains only the features  you must predict the cover type for every row in the test set  565892 observations  '

: 

In [ ]:
nltk.sent_tokenize(text)


['   the goal  the source for this competition is    https   www kaggle com c learn together data  the study area includes four wilderness areas located in the roosevelt national forest of northern colorado  each observation is a 30m x 30m patch  our goal is to predict an integer classification for the forest cover type  the seven types are   1   spruce fir  2   lodgepole pine  3   ponderosa pine  4   cottonwood willow  5   aspen  6   douglas fir  7   krummholz  the training set  15120 observations  contains both features and the cover type  the test set contains only the features  you must predict the cover type for every row in the test set  565892 observations']

: 

In [ ]:
words=nltk.word_tokenize(text)
pprint.pprint(words[:10])

['the',
 'goal',
 'the',
 'source',
 'for',
 'this',
 'competition',
 'is',
 'https',
 'www']


: 

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/kak/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

: 

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

: 

In [ ]:
words = [w for w in words if w not in stopwords.words("english")]
words[:20]

['goal',
 'source',
 'competition',
 'https',
 'www',
 'kaggle',
 'com',
 'c',
 'learn',
 'together',
 'data',
 'study',
 'area',
 'includes',
 'four',
 'wilderness',
 'areas',
 'located',
 'roosevelt',
 'national']

: 

In [ ]:
from nltk.stem.porter import PorterStemmer
# Reduce words to their stems
stemmed = [PorterStemmer().stem(w) for w in words]
print(stemmed)

['goal', 'sourc', 'competit', 'http', 'www', 'kaggl', 'com', 'c', 'learn', 'togeth', 'data', 'studi', 'area', 'includ', 'four', 'wilder', 'area', 'locat', 'roosevelt', 'nation', 'forest', 'northern', 'colorado', 'observ', '30m', 'x', '30m', 'patch', 'goal', 'predict', 'integ', 'classif', 'forest', 'cover', 'type', 'seven', 'type', '1', 'spruce', 'fir', '2', 'lodgepol', 'pine', '3', 'ponderosa', 'pine', '4', 'cottonwood', 'willow', '5', 'aspen', '6', 'dougla', 'fir', '7', 'krummholz', 'train', 'set', '15120', 'observ', 'contain', 'featur', 'cover', 'type', 'test', 'set', 'contain', 'featur', 'must', 'predict', 'cover', 'type', 'everi', 'row', 'test', 'set', '565892', 'observ']


: 